AESO DATA CLEANING AND LOADING

In [41]:
#Combination of imports taken from my ENSF 612 Assignment 2, 3 and Midterm.
#Added a few as datetime was specficially needed for the context of this assignment.
from datetime import datetime, date

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, IntegerType
from pyspark.sql import functions as F
from pyspark.ml import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.ml.feature import (
    StringIndexer, OneHotEncoder, Tokenizer, StopWordsRemover,
    HashingTF, IDF, VectorAssembler, StandardScaler
)

from pyspark.sql import functions as F
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, Bucketizer
from pyspark.ml.classification import DecisionTreeClassifier, LogisticRegression
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator

from pyspark.sql.functions import col

from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import pandas as pd

As we learned in 612, we now want to use SPARK DATAFRAMES to transform our AESO LOAD DATA  
Now that we have already done our Weather Data (using pandas), we decided to take our AESO Load Data, which has columns for each region and than the date and hour as the first column populated and than for each column of the Aeso region has the load amount, ideally we want to be able to join that to the date of our Weather data of each region for that time (for the two features of Temprature and Precipitation) and the of course our Rural and Urban diagnostics for each region which we also created a manual Excel of which we got from research (link provided in that section).


In [42]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("612FinalProject")
    .getOrCreate()
)



25/12/05 07:30:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [43]:
# First Step will be to Read Excel via pandas, then convert to Spark DF
import pandas as pd
import os

print("CWD:", os.getcwd())  # just to confirm we're in .../612FinalProjectGroup3/notebooks

aeso_pd = pd.read_excel(
    "../data/LoadDataAesoRaw/Hourly-load-by-area-and-region-Nov-2023-to-Dec-2024.xlsx",
    engine="openpyxl"
)

aeso_df = spark.createDataFrame(aeso_pd)

aeso_df.printSchema()
aeso_df.show(5)


CWD: /home/behzad/612FinalProjectGroup3/notebooks
root
 |-- DT_MST: timestamp (nullable = true)
 |-- AREA13: double (nullable = true)
 |-- AREA17: double (nullable = true)
 |-- AREA18: double (nullable = true)
 |-- AREA19: double (nullable = true)
 |-- AREA20: double (nullable = true)
 |-- AREA21: double (nullable = true)
 |-- AREA22: double (nullable = true)
 |-- AREA23: double (nullable = true)
 |-- AREA24: double (nullable = true)
 |-- AREA25: double (nullable = true)
 |-- AREA26: double (nullable = true)
 |-- AREA27: double (nullable = true)
 |-- AREA28: double (nullable = true)
 |-- AREA29: double (nullable = true)
 |-- AREA30: double (nullable = true)
 |-- AREA31: double (nullable = true)
 |-- AREA32: double (nullable = true)
 |-- AREA33: double (nullable = true)
 |-- AREA34: double (nullable = true)
 |-- AREA35: double (nullable = true)
 |-- AREA36: double (nullable = true)
 |-- AREA37: double (nullable = true)
 |-- AREA38: double (nullable = true)
 |-- AREA39: double (nullable 

In [44]:
from pyspark.sql.functions import expr, regexp_extract

# This is the actuall extraction of our area columns, so we can use them effectively
area_cols = [c for c in aeso_df.columns if c.startswith("AREA")]
#showing them out to ensure they match.
area_cols


['AREA13',
 'AREA17',
 'AREA18',
 'AREA19',
 'AREA20',
 'AREA21',
 'AREA22',
 'AREA23',
 'AREA24',
 'AREA25',
 'AREA26',
 'AREA27',
 'AREA28',
 'AREA29',
 'AREA30',
 'AREA31',
 'AREA32',
 'AREA33',
 'AREA34',
 'AREA35',
 'AREA36',
 'AREA37',
 'AREA38',
 'AREA39',
 'AREA4',
 'AREA40',
 'AREA42',
 'AREA43',
 'AREA44',
 'AREA45',
 'AREA46',
 'AREA47',
 'AREA48',
 'AREA49',
 'AREA52',
 'AREA53',
 'AREA54',
 'AREA55',
 'AREA56',
 'AREA57',
 'AREA6',
 'AREA60']

In [45]:
# 2 Making a stack of all the areas, which we will use fror getting all these names
# will make our life much easier for the actual working

stack_pairs = ", ".join([f"'{c}', {c}" for c in area_cols])
stack_expr = f"stack({len(area_cols)}, {stack_pairs}) as (area_name, load_mw)"

print(stack_expr)  #This is to ensure it matches


stack(42, 'AREA13', AREA13, 'AREA17', AREA17, 'AREA18', AREA18, 'AREA19', AREA19, 'AREA20', AREA20, 'AREA21', AREA21, 'AREA22', AREA22, 'AREA23', AREA23, 'AREA24', AREA24, 'AREA25', AREA25, 'AREA26', AREA26, 'AREA27', AREA27, 'AREA28', AREA28, 'AREA29', AREA29, 'AREA30', AREA30, 'AREA31', AREA31, 'AREA32', AREA32, 'AREA33', AREA33, 'AREA34', AREA34, 'AREA35', AREA35, 'AREA36', AREA36, 'AREA37', AREA37, 'AREA38', AREA38, 'AREA39', AREA39, 'AREA4', AREA4, 'AREA40', AREA40, 'AREA42', AREA42, 'AREA43', AREA43, 'AREA44', AREA44, 'AREA45', AREA45, 'AREA46', AREA46, 'AREA47', AREA47, 'AREA48', AREA48, 'AREA49', AREA49, 'AREA52', AREA52, 'AREA53', AREA53, 'AREA54', AREA54, 'AREA55', AREA55, 'AREA56', AREA56, 'AREA57', AREA57, 'AREA6', AREA6, 'AREA60', AREA60) as (area_name, load_mw)


In [46]:
#This is no taking the timestamp and the stakc that we made above to format our table with the timestamp
#eseentially we should have each and every area for each time stamp
aeso_long = (
    aeso_df
    .selectExpr("DT_MST as timestamp", stack_expr)
    .where("load_mw is not null")        # drop empty cells if any
)

aeso_long.printSchema()
aeso_long.show(5)

#okay whicked timestamp is a timepsamp format, but I see now everything is aggregated at the first time stamp
#essentially sorted that way, which is good (didnt knwo for sure that was going to happen

root
 |-- timestamp: timestamp (nullable = true)
 |-- area_name: string (nullable = true)
 |-- load_mw: double (nullable = true)

+-------------------+---------+----------+
|          timestamp|area_name|   load_mw|
+-------------------+---------+----------+
|2023-11-01 00:00:00|   AREA13|100.173998|
|2023-11-01 00:00:00|   AREA17|  5.166425|
|2023-11-01 00:00:00|   AREA18|37.6438593|
|2023-11-01 00:00:00|   AREA19|  71.10166|
|2023-11-01 00:00:00|   AREA20| 275.33804|
+-------------------+---------+----------+
only showing top 5 rows


In [47]:
from pyspark.sql.functions import regexp_extract

#Now uisng regexp_extract to take a part the actual numeric part of the area_name from our AESO DATA
#this is huge as it prevents hard coding, always me to demonstarte knowledge of regexp and then we have this as just numerical
#which will serve as the key for our furture join with our WEATHER Data

aeso_long = (
    aeso_long
    .withColumn("area_code",
                regexp_extract("area_name", r"AREA(\d+)", 1).cast("int"))
)

#selecting the specific columns of interest from AESO at this point to show (just the added area_code)
aeso_long.select("timestamp", "area_name", "area_code", "load_mw").show(10)


+-------------------+---------+---------+-----------+
|          timestamp|area_name|area_code|    load_mw|
+-------------------+---------+---------+-----------+
|2023-11-01 00:00:00|   AREA13|       13| 100.173998|
|2023-11-01 00:00:00|   AREA17|       17|   5.166425|
|2023-11-01 00:00:00|   AREA18|       18| 37.6438593|
|2023-11-01 00:00:00|   AREA19|       19|   71.10166|
|2023-11-01 00:00:00|   AREA20|       20|  275.33804|
|2023-11-01 00:00:00|   AREA21|       21|   45.18537|
|2023-11-01 00:00:00|   AREA22|       22|   57.69576|
|2023-11-01 00:00:00|   AREA23|       23|   18.43956|
|2023-11-01 00:00:00|   AREA24|       24| 85.4734143|
|2023-11-01 00:00:00|   AREA25|       25|786.5661492|
+-------------------+---------+---------+-----------+
only showing top 10 rows


In [48]:
import pandas as pd

#now it's time to load of our custom csv which indidcates each region as urban or rural 
#our reasoning and justification can be found in our report
#going to also join this based on area code
mapping_pd = pd.read_csv("../data/area_region_mapping.csv")
mapping_df = spark.createDataFrame(mapping_pd)

mapping_df.printSchema()
mapping_df.show(60)


root
 |-- area_code: long (nullable = true)
 |-- region_type: string (nullable = true)
 |-- location_name: string (nullable = true)

+---------+-----------+--------------------+
|area_code|region_type|       location_name|
+---------+-----------+--------------------+
|        4|      rural|        Medicine Hat|
|        6|      urban|             Calgary|
|       13|      rural|        Lloydminster|
|       17|      rural|        Rainbow Lake|
|       18|      rural|          High Level|
|       19|      rural|         Peace River|
|       20|      rural|      Grande Prairie|
|       21|      rural|        High Prairie|
|       22|      rural|        Grande Cache|
|       23|      rural|          Valleyview|
|       24|      rural|           Fox Creek|
|       25|      rural|       Fort McMurray|
|       26|      rural|          Swan Hills|
|       27|      rural|Athabasca/Lac La ...|
|       28|      rural|           Cold Lake|
|       29|      rural|        Hinton/Edson|
|       30| 

In [49]:
#Now time for our first merge, before weather we are joining the region to each of our data points
#based on the area code -> we did this lots in 612, so this was straightforward with the join
#on and of course inner -> beauty of pyspark and the allowing of pseudo SQL commands

aeso_with_region = (
    aeso_long
    .join(mapping_df, on="area_code", how="inner")
)

aeso_with_region.select(
    "timestamp", "area_code", "area_name",
    "location_name", "region_type", "load_mw"
)

aeso_with_region.show(50)

+---------+-------------------+---------+--------+-----------+-------------+
|area_code|          timestamp|area_name| load_mw|region_type|location_name|
+---------+-------------------+---------+--------+-----------+-------------+
|       19|2023-11-01 00:00:00|   AREA19|71.10166|      rural|  Peace River|
|       19|2023-11-01 01:00:00|   AREA19| 71.0077|      rural|  Peace River|
|       19|2023-11-01 02:00:00|   AREA19|70.51349|      rural|  Peace River|
|       19|2023-11-01 03:00:00|   AREA19|70.87265|      rural|  Peace River|
|       19|2023-11-01 04:00:00|   AREA19|71.12218|      rural|  Peace River|
|       19|2023-11-01 05:00:00|   AREA19|73.80247|      rural|  Peace River|
|       19|2023-11-01 06:00:00|   AREA19| 79.6731|      rural|  Peace River|
|       19|2023-11-01 07:00:00|   AREA19|83.85579|      rural|  Peace River|
|       19|2023-11-01 08:00:00|   AREA19|83.11333|      rural|  Peace River|
|       19|2023-11-01 09:00:00|   AREA19|82.57626|      rural|  Peace River|

In [50]:
from pyspark.sql.functions import sum as F_sum

#Grouping by region type, thought this would be cool to look at so essentially just reshows each timeperiod as urban or rural instead of area,
# perhaps not super valuable but not bad.
urban_rural_long = (
    aeso_with_region
    .groupBy("timestamp", "region_type")
    .agg(F_sum("load_mw").alias("total_load_mw"))
)

urban_rural_long.show(5)
urban_rural_long.printSchema()


+-------------------+-----------+------------------+
|          timestamp|region_type|     total_load_mw|
+-------------------+-----------+------------------+
|2023-11-01 13:00:00|      rural|       3091.182386|
|2023-11-05 22:00:00|      rural|      3210.6321512|
|2023-11-12 00:00:00|      urban|3290.9197912000004|
|2023-11-26 20:00:00|      urban|4010.1097236000005|
|2023-12-03 23:00:00|      urban|      3482.9017276|
+-------------------+-----------+------------------+
only showing top 5 rows
root
 |-- timestamp: timestamp (nullable = true)
 |-- region_type: string (nullable = true)
 |-- total_load_mw: double (nullable = true)



In [51]:
from pyspark.sql.functions import first

#Grouping by region type, thought this would be cool to look at so essentially combining every area for each time
#into urban and rural, not sure if we will use this past this but not bad

urban_rural = (
    urban_rural_long
    .groupBy("timestamp")
    .pivot("region_type", ["urban", "rural"])  # order the columns explicitly
    .agg(first("total_load_mw"))
    .orderBy("timestamp")
)

urban_rural.show(10)
urban_rural.printSchema()


+-------------------+------------------+------------------+
|          timestamp|             urban|             rural|
+-------------------+------------------+------------------+
|2023-11-01 00:00:00|      3201.0208318|2992.1147807999996|
|2023-11-01 01:00:00|3126.1436237999997|2955.5653366999995|
|2023-11-01 02:00:00|      3145.5424359|      2981.0202757|
|2023-11-01 03:00:00|3189.7622247000004|2969.2898119999995|
|2023-11-01 04:00:00|      3323.4024472|3011.3032270000003|
|2023-11-01 05:00:00|3669.7011083999996|      3075.4196297|
|2023-11-01 06:00:00|3983.8241967000004|      3167.2820205|
|2023-11-01 07:00:00|      4128.9717709|      3149.9376985|
|2023-11-01 08:00:00|      4116.7720527|      3148.5074315|
|2023-11-01 09:00:00|4057.8694981999997|      3141.5562506|
+-------------------+------------------+------------------+
only showing top 10 rows
root
 |-- timestamp: timestamp (nullable = true)
 |-- urban: double (nullable = true)
 |-- rural: double (nullable = true)



In [52]:
from pyspark.sql.functions import hour, dayofweek, month

#okay this where we are actually adding features, I knew I wanted to achieve this and with some help from CHATGPT
#using the import of hour day of the week and month this turned out beautiful, these are categorical featurs
#in my eyes and allows for encoding and vectorizing features to help our model!!!!!
#as day of the week is super important, I think esepcially in the scope of Energy usage right, like wweekend vs holiday, so the fact
#that we have this is great, and same for month and lastly hour -> helps understand if people are home or not ect, so these
#are a bunch of of great features to add to our data set.

#doing this on our tester urban_rural_features -> this is not our main dataframe which will do next
urban_rural_features = (
    urban_rural
    .withColumn("hour", hour("timestamp"))
    .withColumn("dayofweek", dayofweek("timestamp"))  # This is directly from pypark.sql.functions, upon doing more into this librayr
        #Sunday Maps to 1 and Saturday maps to 7, so this confroms that November 1st 2023 was a Wednesday.
    .withColumn("month", month("timestamp"))
)

urban_rural_features.show(10)


+-------------------+------------------+------------------+----+---------+-----+
|          timestamp|             urban|             rural|hour|dayofweek|month|
+-------------------+------------------+------------------+----+---------+-----+
|2023-11-01 00:00:00|      3201.0208318|2992.1147807999996|   0|        4|   11|
|2023-11-01 01:00:00|3126.1436237999997|2955.5653366999995|   1|        4|   11|
|2023-11-01 02:00:00|      3145.5424359|      2981.0202757|   2|        4|   11|
|2023-11-01 03:00:00|3189.7622247000004|2969.2898119999995|   3|        4|   11|
|2023-11-01 04:00:00|      3323.4024472|3011.3032270000003|   4|        4|   11|
|2023-11-01 05:00:00|3669.7011083999996|      3075.4196297|   5|        4|   11|
|2023-11-01 06:00:00|3983.8241967000004|      3167.2820205|   6|        4|   11|
|2023-11-01 07:00:00|      4128.9717709|      3149.9376985|   7|        4|   11|
|2023-11-01 08:00:00|      4116.7720527|      3148.5074315|   8|        4|   11|
|2023-11-01 09:00:00|4057.86

Okay this all working beautiful as you can see above  

This is exactly what we wanted as we have

Lastly we want to add Day of The Week, Month & Season. To Aeso with region

In [98]:
from pyspark.sql.functions import hour, dayofweek, month, when, col

#Now we want to what we just did but on our working Dataframe which is in the form we wanted
#first will add hour, month day of week, but also SEASON!! this is another cool add that I wanted to see and
#could be of importance.
aeso_time_features = (
    aeso_with_region
        # 1) Hour of day (0–23)
        .withColumn("hour", hour("timestamp"))

        # 2) Day of week (Spark: 1 = Sunday, ..., 7 = Saturday)
        .withColumn("dayofweek", dayofweek("timestamp"))

        # 3) Month number (1 = January, ..., 12 = December)
        .withColumn("month", month("timestamp"))

        # 4) Season (meteorological seasons for Alberta / N. hemisphere)
        .withColumn(
            "season",
            when(col("month").isin(12, 1, 2),  "winter")
            .when(col("month").isin(3, 4, 5),  "spring")
            .when(col("month").isin(6, 7, 8),  "summer")
            .otherwise("fall")  # 9,10,11
        )
)

#now we can see all our features together and looking not bad at all, this is quite good for our AESO data
#we will start here for our models, and then see if we can get creative and add more features.
aeso_time_features.select(
    "timestamp", "area_code", "location_name", "region_type",
    "hour", "dayofweek", "month", "season", "load_mw"
).show(10)

+-------------------+---------+-------------+-----------+----+---------+-----+------+--------+
|          timestamp|area_code|location_name|region_type|hour|dayofweek|month|season| load_mw|
+-------------------+---------+-------------+-----------+----+---------+-----+------+--------+
|2023-11-01 00:00:00|       19|  Peace River|      rural|   0|        4|   11|  fall|71.10166|
|2023-11-01 01:00:00|       19|  Peace River|      rural|   1|        4|   11|  fall| 71.0077|
|2023-11-01 02:00:00|       19|  Peace River|      rural|   2|        4|   11|  fall|70.51349|
|2023-11-01 03:00:00|       19|  Peace River|      rural|   3|        4|   11|  fall|70.87265|
|2023-11-01 04:00:00|       19|  Peace River|      rural|   4|        4|   11|  fall|71.12218|
|2023-11-01 05:00:00|       19|  Peace River|      rural|   5|        4|   11|  fall|73.80247|
|2023-11-01 06:00:00|       19|  Peace River|      rural|   6|        4|   11|  fall| 79.6731|
|2023-11-01 07:00:00|       19|  Peace River|     

In [99]:
#Okay now that our first script we made our full weather data frame and wrote it to a final CSV
# we can make a spark for it, so we have and can join with our spark data frame of AESO

#and we will be in great shape to build our first model

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp


spark = (
    SparkSession.builder
    .appName("612 Final Project - Combine AESO & Weather")
    .getOrCreate()
)

weather_spark = (
    spark.read
        .option("header", True)
        .csv("../data/weather_all_areas_hourly_with_daylight.csv")
)

weather_spark.printSchema()
weather_spark.show(5)

root
 |-- area_code: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- temp_c: string (nullable = true)
 |-- precip_mm: string (nullable = true)
 |-- daylight: string (nullable = true)

+---------+-------------------+------+---------+--------+
|area_code|          timestamp|temp_c|precip_mm|daylight|
+---------+-------------------+------+---------+--------+
|        4|2023-11-01 00:00:00|  -4.6|      0.0|       0|
|        4|2023-11-01 01:00:00|  -5.9|      0.0|       0|
|        4|2023-11-01 02:00:00|  -6.5|      0.0|       0|
|        4|2023-11-01 03:00:00|  -6.9|      0.0|       0|
|        4|2023-11-01 04:00:00|  -6.8|      0.0|       0|
+---------+-------------------+------+---------+--------+
only showing top 5 rows


In [100]:
#realized from our schema above that our TIMESTAMP was a string not a TIMESTAMP, so renamed above BAD,
#now we do casting to ensure our schema is perfect
#this will be critical for our joining as we are doing by both TIMESTAMPE and of Course Area_Code

weather_spark_cleaned = (
    weather_spark
        .withColumn("area_code",  col("area_code").cast("int"))
        .withColumn("timestamp",  to_timestamp("timestamp"))
        .withColumn("temp_c",     col("temp_c").cast("double"))
        .withColumn("precip_mm",  col("precip_mm").cast("double"))
        .withColumn("daylight",  col("daylight").cast("int"))
)

weather_spark_cleaned.printSchema()
weather_spark_cleaned.show(5)

root
 |-- area_code: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- temp_c: double (nullable = true)
 |-- precip_mm: double (nullable = true)
 |-- daylight: integer (nullable = true)

+---------+-------------------+------+---------+--------+
|area_code|          timestamp|temp_c|precip_mm|daylight|
+---------+-------------------+------+---------+--------+
|        4|2023-11-01 00:00:00|  -4.6|      0.0|       0|
|        4|2023-11-01 01:00:00|  -5.9|      0.0|       0|
|        4|2023-11-01 02:00:00|  -6.5|      0.0|       0|
|        4|2023-11-01 03:00:00|  -6.9|      0.0|       0|
|        4|2023-11-01 04:00:00|  -6.8|      0.0|       0|
+---------+-------------------+------+---------+--------+
only showing top 5 rows


In [101]:
#This is the crux, which the join, which will we do on each timestampe(now that they match) and the area code,
# so after this another column for both precipiation and temprature will be added for each row respective to it

#took a ton of coordination, preparation and manipulation, but it looks great!! #exactly as intended

aeso_weather = (
    aeso_time_features
        .join(
            weather_spark,
            on=["timestamp", "area_code"],  
            how="inner"                     
        )
)

aeso_weather.printSchema()
aeso_weather.show(10)

root
 |-- timestamp: timestamp (nullable = true)
 |-- area_code: integer (nullable = true)
 |-- area_name: string (nullable = true)
 |-- load_mw: double (nullable = true)
 |-- region_type: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- season: string (nullable = false)
 |-- temp_c: string (nullable = true)
 |-- precip_mm: string (nullable = true)
 |-- daylight: string (nullable = true)



+-------------------+---------+---------+------------+-----------+-----------------+----+---------+-----+------+------+---------+--------+
|          timestamp|area_code|area_name|     load_mw|region_type|    location_name|hour|dayofweek|month|season|temp_c|precip_mm|daylight|
+-------------------+---------+---------+------------+-----------+-----------------+----+---------+-----+------+------+---------+--------+
|2023-11-01 00:00:00|       29|   AREA29|  84.1644369|      rural|     Hinton/Edson|   0|        4|   11|  fall|  -2.8|      0.0|       0|
|2023-11-01 00:00:00|       44|   AREA44| 137.9625539|      urban|            Seebe|   0|        4|   11|  fall|  -1.2|      0.0|       0|
|2023-11-01 00:00:00|       49|   AREA49|    9.393875|      rural|          Stavely|   0|        4|   11|  fall|  -5.1|      0.0|       0|
|2023-11-01 00:00:00|       60|   AREA60|1123.2788268|      urban|         Edmonton|   0|        4|   11|  fall|   0.8|      0.0|       0|
|2023-11-01 02:00:00|      

# FIXING SCHEMA

Good thing I checked the schema, because our temp_c and precip_m are STRINGS
That was a big miss, dint even realize until vector assembly and got an error as you can only take in numeric or vector columns
WHole reason for encoding, so I will just type cast them as doubles here

In [102]:

aeso_weather = (
    aeso_weather
        .withColumn("temp_c", col("temp_c").cast("double"))
        .withColumn("precip_mm", col("precip_mm").cast("double"))
        .withColumn("daylight", col("daylight").cast("int"))  # ADDED: Ensure daylight is int
)

aeso_weather.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- area_code: integer (nullable = true)
 |-- area_name: string (nullable = true)
 |-- load_mw: double (nullable = true)
 |-- region_type: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- season: string (nullable = false)
 |-- temp_c: double (nullable = true)
 |-- precip_mm: double (nullable = true)
 |-- daylight: integer (nullable = true)



In [103]:
aeso_weather.printSchema()
aeso_weather.show(10, truncate=False)
aeso_weather.describe(["load_mw", "temp_c", "precip_mm","daylight"]).show()


root
 |-- timestamp: timestamp (nullable = true)
 |-- area_code: integer (nullable = true)
 |-- area_name: string (nullable = true)
 |-- load_mw: double (nullable = true)
 |-- region_type: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- season: string (nullable = false)
 |-- temp_c: double (nullable = true)
 |-- precip_mm: double (nullable = true)
 |-- daylight: integer (nullable = true)



+-------------------+---------+---------+------------+-----------+-----------------+----+---------+-----+------+------+---------+--------+
|timestamp          |area_code|area_name|load_mw     |region_type|location_name    |hour|dayofweek|month|season|temp_c|precip_mm|daylight|
+-------------------+---------+---------+------------+-----------+-----------------+----+---------+-----+------+------+---------+--------+
|2023-11-01 00:00:00|29       |AREA29   |84.1644369  |rural      |Hinton/Edson     |0   |4        |11   |fall  |-2.8  |0.0      |0       |
|2023-11-01 00:00:00|44       |AREA44   |137.9625539 |urban      |Seebe            |0   |4        |11   |fall  |-1.2  |0.0      |0       |
|2023-11-01 00:00:00|49       |AREA49   |9.393875    |rural      |Stavely          |0   |4        |11   |fall  |-5.1  |0.0      |0       |
|2023-11-01 00:00:00|60       |AREA60   |1123.2788268|urban      |Edmonton         |0   |4        |11   |fall  |0.8   |0.0      |0       |
|2023-11-01 02:00:00|4     

+-------+-----------------+------------------+------------------+-------------------+
|summary|          load_mw|            temp_c|         precip_mm|           daylight|
+-------+-----------------+------------------+------------------+-------------------+
|  count|           426840|            416592|            362015|             426840|
|   mean|165.8880732508191|2.9598772900103727|0.0408850461997432|0.43811732733576986|
| stddev|272.9519930960776|12.370707610516746|0.3452698979226287|  0.496156337814179|
|    min|              0.0|             -47.4|               0.0|                  0|
|    max|      2121.356533|              38.0|              44.4|                  1|
+-------+-----------------+------------------+------------------+-------------------+



In [104]:
from pyspark.sql.functions import col, sum as F_sum

aeso_weather.select([
    F_sum(col(c).isNull().cast("int")).alias(c + "_nulls")
    for c in ["load_mw", "temp_c", "precip_mm", "daylight", "area_code", "timestamp"]
]).show()


+-------------+------------+---------------+--------------+---------------+---------------+
|load_mw_nulls|temp_c_nulls|precip_mm_nulls|daylight_nulls|area_code_nulls|timestamp_nulls|
+-------------+------------+---------------+--------------+---------------+---------------+
|            0|       10248|          64825|             0|              0|              0|
+-------------+------------+---------------+--------------+---------------+---------------+



# DEALING WITH NULLS FOR FINAL DATA SET

So we realized we NULLS in both temp_c_nulls and precip_mm_nulls, so the before we can do any modelling we should deal with them.

Of course in 612 Week 6 we learned a ton of techniques to deal with that, and will apply one as such.

After deliberation I think we realized that for precipitation, that rain is more random than temprature, just because it rained another day that month isnt neccesarily indicative that is raining again, what we think is the best strategy as when it rains (it rains for generally a while), is to fill nulls with the average precipation of the whole (average of every hour) in that SPECIFIC REGION and replace the rain for the null hours with that.

In [105]:
from pyspark.sql.functions import to_date

aeso_weather_day = (
    aeso_weather
        .withColumn("day", to_date("timestamp"))  # e.g. 2023-11-01
)

In [106]:
from pyspark.sql import Window
from pyspark.sql.functions import avg, when, col

w_daily = Window.partitionBy("area_code", "day")

aeso_weather_precip_filled = (
    aeso_weather_day
        # average of non-null precip_mm for that region+day
        .withColumn("daily_precip_avg", avg("precip_mm").over(w_daily))
        # fill only where precip_mm is null
        .withColumn(
            "precip_mm_filled",
            when(col("precip_mm").isNull(), col("daily_precip_avg"))
            .otherwise(col("precip_mm"))
        )
        .drop("daily_precip_avg")  # optional cleanup
)

In [107]:
aeso_weather_clean_precipation = (
    aeso_weather_precip_filled
        .drop("precip_mm")
        .withColumnRenamed("precip_mm_filled", "precip_mm")
)

In [108]:
#Rime to check
aeso_weather_clean_precipation.select([
    F_sum(col(c).isNull().cast("int")).alias(c + "_nulls")
    for c in ["load_mw", "temp_c", "precip_mm", "area_code", "timestamp"]
]).show()

from pyspark.sql.functions import col



+-------------+------------+---------------+---------------+---------------+
|load_mw_nulls|temp_c_nulls|precip_mm_nulls|area_code_nulls|timestamp_nulls|
+-------------+------------+---------------+---------------+---------------+
|            0|       10248|          64824|              0|              0|
+-------------+------------+---------------+---------------+---------------+



As that had almost no impact, what this now means is that most likely we just dont have data for a set period so there is nothing in that day to take the avergae from and we cant replace NULLS,  

We need to inspect further so my next step was to see where the nulls actually live (in which areas and then I can do an intentional exploration on the data at hand.)

In [109]:
# look at distinct (area_code, month) where precip_c is null
aeso_weather.filter(col("precip_mm").isNull()) \
    .select("area_code", "month") \
    .distinct() \
    .show()

+---------+-----+
|area_code|month|
+---------+-----+
|       19|    2|
|       26|    4|
|       26|   12|
|       26|    1|
|       19|    1|
|       19|    4|
|       22|   11|
|       26|   11|
|       19|    6|
|       26|    3|
|       22|   10|
|       19|    7|
|       26|   10|
|       19|   12|
|       19|    9|
|       19|    3|
|       19|    5|
|       26|    2|
|       19|    8|
|       19|   10|
+---------+-----+
only showing top 20 rows


# WHAT THIS CHECK TELLS US ABOUT NULLS
This tells us it is not a bad weather station issue with corrupt data there are nulls across many different weather stations without a specific concentration in one, after further inspection into the raw data of the stations of interest (connected to the area code provided), this was confiemed

Lets go back to our original data set and just fill them with 0, thats also probably pretty safe when it comes to rain.

In [110]:
aeso_weather = aeso_weather.fillna({
    "precip_mm": 0.0,
    "daylight": 0  
})

#check
aeso_weather.select([
    F_sum(col(c).isNull().cast("int")).alias(c + "_nulls")
    for c in ["load_mw", "temp_c", "precip_mm", "area_code", "timestamp"]
]).show()

+-------------+------------+---------------+---------------+---------------+
|load_mw_nulls|temp_c_nulls|precip_mm_nulls|area_code_nulls|timestamp_nulls|
+-------------+------------+---------------+---------------+---------------+
|            0|       10248|              0|              0|              0|
+-------------+------------+---------------+---------------+---------------+



# SUMMARY TIME OF NUMERICAL AND CATEGORICAL COLUMNS BEFORE MODELLING

Just as we leaned (and taken from our work in Assignment 3), I learned its great to look first at both our Numerical and Categorical Columns (which of course will indexed and vectorized), before building our final pipeline to predict Load in MW


In [111]:
#first of course we want to predict load for a certain timestamp, so we can extend that to the future (not including the year),

label_col = "load_mw"

In [112]:
numeric_cols = ['temp_c', 'precip_mm', 'daylight']

aeso_weather.describe(numeric_cols).show()

+-------+------------------+-------------------+-------------------+
|summary|            temp_c|          precip_mm|           daylight|
+-------+------------------+-------------------+-------------------+
|  count|            416592|             426840|             426840|
|   mean| 2.959877290010362|0.03467575672383007|0.43811732733576986|
| stddev|12.370707610516764| 0.3183107378239392| 0.4961563378141771|
|    min|             -47.4|                0.0|                  0|
|    max|              38.0|               44.4|                  1|
+-------+------------------+-------------------+-------------------+



In [113]:
#Nowe lets do categorical, NOW NOTE (even though month, day and hour represented by inetgers truly they are
#categories as they can only be a finite value and month and day can be replaced by string i.e November or Saturday, so I chose to leave
#them as categories due to what they represent:
categorical_cols = ["area_code","region_type", "hour", "dayofweek", "month", "season"]


for column in categorical_cols:
        stats = (
        aeso_weather
        .groupby(column)
        .agg(F.count("*").alias("n_rows"),F.avg("load_mw").alias("avg_load_mw"),F.min("load_mw").alias("min_load_mw"),
            F.max("load_mw").alias("max_load_mw")).orderBy(column))
        stats.show(truncate=False) # if using DB i realized we display(stats)

+---------+------+------------------+-----------+------------+
|area_code|n_rows|avg_load_mw       |min_load_mw|max_load_mw |
+---------+------+------------------+-----------+------------+
|4        |10248 |27.30073670490833 |4.1753448  |55.2867376  |
|6        |10248 |1117.9211637504688|764.6293749|1824.1138584|
|13       |10248 |88.38278668826173 |50.8265535 |137.8933409 |
|17       |10248 |12.041962755679217|2.60726    |47.3053119  |
|18       |10248 |36.178930407572125|14.3762844 |53.6026496  |
|19       |10248 |73.00554772053106 |50.07674   |106.47881   |
|20       |10248 |280.4811968871972 |208.3936   |348.37712   |
|21       |10248 |76.97281901053846 |33.95139   |101.16912   |
|22       |10248 |54.940816131928045|11.06284   |69.33908    |
|23       |10248 |23.719194094457514|9.89584    |35.2212     |
|24       |10248 |82.96259741082159 |43.5704975 |99.157371   |
|25       |10248 |688.6553386539241 |498.1173409|898.8798419 |
|26       |10248 |197.20513409236   |66.9230553 |287.67

+-----------+------+------------------+-----------+-----------+
|region_type|n_rows|avg_load_mw       |min_load_mw|max_load_mw|
+-----------+------+------------------+-----------+-----------+
|rural      |283368|112.59638268266158|0.0        |898.8798419|
|urban      |143472|271.14317370887125|2.9328187  |2121.356533|
+-----------+------+------------------+-----------+-----------+



+----+------+------------------+-----------+------------+
|hour|n_rows|avg_load_mw       |min_load_mw|max_load_mw |
+----+------+------------------+-----------+------------+
|0   |17785 |153.91109948992423|0.0        |1519.0256447|
|1   |17785 |151.71338099371965|0.0        |1501.087999 |
|2   |17743 |150.65805853058126|0.0        |1450.9684422|
|3   |17827 |150.6630734374993 |0.0        |1451.388363 |
|4   |17785 |152.13900690899646|0.0        |1461.1570141|
|5   |17785 |156.26909920314864|0.0        |1498.7311083|
|6   |17785 |162.30985143858317|0.0        |1591.7538483|
|7   |17785 |166.98960630035432|0.0        |1719.9936441|
|8   |17785 |168.98173228442508|0.0        |1803.3920744|
|9   |17785 |169.82228544831605|0.0        |1894.9085384|
|10  |17785 |170.71994723980333|0.0        |1962.4180801|
|11  |17785 |171.09345309515876|0.0        |2001.2324488|
|12  |17785 |171.24440111788016|0.0        |2052.9704715|
|13  |17785 |171.24737067584493|0.0        |2082.5886139|
|14  |17785 |1

+---------+------+------------------+-----------+------------+
|dayofweek|n_rows|avg_load_mw       |min_load_mw|max_load_mw |
+---------+------+------------------+-----------+------------+
|1        |60984 |161.30989365334995|0.0        |2007.9945971|
|2        |60960 |167.34958801058048|0.0        |2102.694936 |
|3        |60960 |168.08662808255391|0.0        |2009.6570069|
|4        |60984 |167.8324009797488 |0.0        |2121.356533 |
|5        |60984 |167.69072080252528|0.0        |2107.0080031|
|6        |60984 |167.0799176555013 |0.0        |2034.2196704|
|7        |60984 |161.86880397666485|0.0        |1903.0245662|
+---------+------+------------------+-----------+------------+



+-----+------+------------------+-----------+------------+
|month|n_rows|avg_load_mw       |min_load_mw|max_load_mw |
+-----+------+------------------+-----------+------------+
|1    |31248 |176.16515389315788|0.0        |1892.5002711|
|2    |29232 |168.5755355338325 |0.0        |1629.8547005|
|3    |31248 |164.68070042870283|0.0        |1621.8241125|
|4    |30240 |157.6248427067362 |0.0        |1493.1059788|
|5    |30624 |155.28632600689022|0.0        |1533.9919309|
|6    |29520 |159.57220848143282|0.0        |1569.0155166|
|7    |30504 |176.15465473888668|0.0        |2121.356533 |
|8    |30504 |164.79456935265216|0.0        |1824.4598154|
|9    |29520 |158.19389247204245|0.0        |1800.8761273|
|10   |31224 |159.8397935854342 |0.0        |1569.5191787|
|11   |60480 |168.05167523894215|0.0        |1726.0901032|
|12   |62496 |172.35785841526993|0.0        |1782.333449 |
+-----+------+------------------+-----------+------------+



+------+------+------------------+-----------+------------+
|season|n_rows|avg_load_mw       |min_load_mw|max_load_mw |
+------+------+------------------+-----------+------------+
|fall  |121224|163.53598907095548|0.0        |1800.8761273|
|spring|92112 |159.24099159808497|0.0        |1621.8241125|
|summer|90528 |166.9194804487032 |0.0        |2121.356533 |
|winter|122976|172.42620920422672|0.0        |1892.5002711|
+------+------+------------------+-----------+------------+



In [114]:
#Hmmmm lots of good insights here, will discuss later

# Modelling Time

# Feature Columns Time

In [115]:
#Lets first set up where everything is

#realized that even thoigh area code is a int it's really a category as it respresents an area
#so we have to convert it to a string (its also not complete)

aeso_weather = aeso_weather.withColumn("area_code_str", col("area_code").cast("string"))

numeric_cols = ["temp_c", "precip_mm", "daylight"]

categorical_cols = ["region_type","season","hour","dayofweek","month","area_code"]

In [116]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

indexer = StringIndexer(inputCols= categorical_cols, outputCols= [column + "Index" for column in categorical_cols] )
encoder = OneHotEncoder(inputCols = [column + "Index" for column in categorical_cols], outputCols = [column + "Encoder" for column in categorical_cols])


indexerModel = indexer.fit(aeso_weather)
aeso_weather = indexerModel.transform(aeso_weather)
encoderModel = encoder.fit(aeso_weather)
aeso_weather = encoderModel.transform(aeso_weather)

In [117]:
#Awesome time this, again great reference to this week 7 and 8 notes, esepcially in machine-learning.iypmb you provided

#now need our input columns, so will use my already defined numerical array and than pull these encoded columns (output from encoder, which I will just do manually)

combineInputColumns = numeric_cols + [column + "Encoder" for column in categorical_cols]

print(combineInputColumns)

#Excellent that all looks good!, so can use our assembler
#Actually have to remove price from that, good catch

combineInputColumnsGood = [c for c in combineInputColumns if c != "load_mw"]
print(combineInputColumns)

#great assembler time

assembler = VectorAssembler(inputCols=combineInputColumnsGood, outputCol="features_raw", handleInvalid="skip")
aeso_weather_Assembled = assembler.transform(aeso_weather)




['temp_c', 'precip_mm', 'daylight', 'region_typeEncoder', 'seasonEncoder', 'hourEncoder', 'dayofweekEncoder', 'monthEncoder', 'area_codeEncoder']
['temp_c', 'precip_mm', 'daylight', 'region_typeEncoder', 'seasonEncoder', 'hourEncoder', 'dayofweekEncoder', 'monthEncoder', 'area_codeEncoder']


Beautiful that looks great all we have left is our Numeric columns of Temp and Precip and the rest are the encoded vectorized feature columns with the original (category columns removed as they have now been vectorized)

In [118]:
scaler = StandardScaler(inputCol = "features_raw", outputCol = "features", withMean = False, withStd = True)
scalerModel = scaler.fit(aeso_weather_Assembled)
aeso_weather_Scaled = scalerModel.transform(aeso_weather_Assembled)
aeso_weather_Final = aeso_weather_Scaled

# NOW THAT WE HAVE FINAL DATA SET

In [119]:
#Need to build our test and train sets

In [120]:
train_df, test_df = aeso_weather_Final.randomSplit([0.8, 0.2], seed=42)

In [121]:
train_df, test_df = aeso_weather_Final.randomSplit([0.8, 0.2], seed=42)

#COPIED MY CODE FROM ASSIGNMENT 2 that I wrote to get the calculations
#Now just doing some calculation and holding them in variables not match the loaded ouput

totalRowsPercentage = aeso_weather_Final.count() / aeso_weather.count() * 100
traindDFRowsPercentage = train_df.count() / aeso_weather_Final.count() * 100
testDFRowsPercentage = test_df.count() / aeso_weather_Final.count() * 100

print(f"Total rows: {aeso_weather_Final.count()}, ({totalRowsPercentage:.2f}%)")
print(f"Training rows: {train_df.count()}, ({traindDFRowsPercentage:.2f}%)")
print(f"Testing rows: {test_df.count()}, ({testDFRowsPercentage:.2f}%)")

Total rows: 416592, (97.60%)


Training rows: 333695, (80.10%)


Testing rows: 82897, (19.90%)


In [122]:

decision_tree = DecisionTreeRegressor(labelCol="load_mw", featuresCol="features", predictionCol = "prediction", maxDepth =20, minInstancesPerNode = 10)
decisionTreeModel = decision_tree.fit(train_df)

25/12/05 09:54:14 WARN DAGScheduler: Broadcasting large task binary with size 1059.2 KiB
25/12/05 09:54:15 WARN DAGScheduler: Broadcasting large task binary with size 1190.9 KiB
25/12/05 09:54:15 WARN DAGScheduler: Broadcasting large task binary with size 1335.9 KiB


In [123]:
predictionsNonPipeline = decisionTreeModel.transform(test_df)

In [124]:
predictionsNonPipeline.show(5, truncate=True)

+-------------------+---------+---------+----------+-----------+------------------+----+---------+-----+------+------+---------+--------+-------------+----------------+-----------+---------+--------------+----------+--------------+------------------+-------------+---------------+----------------+--------------+----------------+--------------------+--------------------+------------------+
|          timestamp|area_code|area_name|   load_mw|region_type|     location_name|hour|dayofweek|month|season|temp_c|precip_mm|daylight|area_code_str|region_typeIndex|seasonIndex|hourIndex|dayofweekIndex|monthIndex|area_codeIndex|region_typeEncoder|seasonEncoder|    hourEncoder|dayofweekEncoder|  monthEncoder|area_codeEncoder|        features_raw|            features|        prediction|
+-------------------+---------+---------+----------+-----------+------------------+----+---------+-----+------+------+---------+--------+-------------+----------------+-----------+---------+--------------+----------+--

In [125]:
# Make sure columns look right
print(train_df.columns)

# Compare true vs predicted loads
predictionsNonPipeline.select(
    "timestamp", "area_name", "load_mw", "prediction"
).show(10, truncate=False)

['timestamp', 'area_code', 'area_name', 'load_mw', 'region_type', 'location_name', 'hour', 'dayofweek', 'month', 'season', 'temp_c', 'precip_mm', 'daylight', 'area_code_str', 'region_typeIndex', 'seasonIndex', 'hourIndex', 'dayofweekIndex', 'monthIndex', 'area_codeIndex', 'region_typeEncoder', 'seasonEncoder', 'hourEncoder', 'dayofweekEncoder', 'monthEncoder', 'area_codeEncoder', 'features_raw', 'features']


+-------------------+---------+----------+------------------+
|timestamp          |area_name|load_mw   |prediction        |
+-------------------+---------+----------+------------------+
|2023-11-01 00:00:00|AREA49   |9.393875  |14.421533918656342|
|2023-11-01 02:00:00|AREA53   |11.3977542|19.62312578647055 |
|2023-11-01 03:00:00|AREA46   |43.1809826|64.30154343448073 |
|2023-11-01 04:00:00|AREA45   |68.992275 |64.30154343448073 |
|2023-11-01 07:00:00|AREA31   |118.521359|64.30154343448073 |
|2023-11-01 09:00:00|AREA21   |69.40392  |64.30154343448073 |
|2023-11-01 10:00:00|AREA53   |12.4471908|19.62312578647055 |
|2023-11-01 14:00:00|AREA24   |80.2330875|64.30154343448073 |
|2023-11-01 18:00:00|AREA13   |93.4458097|64.30154343448073 |
|2023-11-01 18:00:00|AREA17   |5.72264   |13.823112301117026|
+-------------------+---------+----------+------------------+
only showing top 10 rows


# Gradient Boosting Model

In [126]:
# Gradient Boosting Model
from pyspark.ml.regression import GBTRegressor

# Create and train Gradient Boosting model
gbt = GBTRegressor(
    labelCol="load_mw", 
    featuresCol="features", 
    predictionCol="prediction",
    maxDepth=10,
    maxIter=50,
    stepSize=0.1
)

gbt_model = gbt.fit(train_df)

# Make predictions
gbt_predictions = gbt_model.transform(test_df)



25/12/05 09:54:34 WARN DAGScheduler: Broadcasting large task binary with size 1033.4 KiB
25/12/05 09:54:34 WARN DAGScheduler: Broadcasting large task binary with size 1030.1 KiB
25/12/05 09:54:34 WARN DAGScheduler: Broadcasting large task binary with size 1030.6 KiB
25/12/05 09:54:34 WARN DAGScheduler: Broadcasting large task binary with size 1031.2 KiB
25/12/05 09:54:34 WARN DAGScheduler: Broadcasting large task binary with size 1032.4 KiB
25/12/05 09:54:35 WARN DAGScheduler: Broadcasting large task binary with size 1034.8 KiB
25/12/05 09:54:35 WARN DAGScheduler: Broadcasting large task binary with size 1039.4 KiB
25/12/05 09:54:35 WARN DAGScheduler: Broadcasting large task binary with size 1048.5 KiB
25/12/05 09:54:35 WARN DAGScheduler: Broadcasting large task binary with size 1065.4 KiB
25/12/05 09:54:35 WARN DAGScheduler: Broadcasting large task binary with size 1096.3 KiB
25/12/05 09:54:35 WARN DAGScheduler: Broadcasting large task binary with size 1150.5 KiB
25/12/05 09:54:35 WAR

In [127]:
# Model Evaluation - Gradient Boosting
from pyspark.ml.evaluation import RegressionEvaluator

# Create evaluator
evaluator = RegressionEvaluator(
    labelCol="load_mw",
    predictionCol="prediction"
)

# Evaluate Gradient Boosting model
gbt_rmse = evaluator.evaluate(gbt_predictions, {evaluator.metricName: "rmse"})
gbt_mae = evaluator.evaluate(gbt_predictions, {evaluator.metricName: "mae"})
gbt_r2 = evaluator.evaluate(gbt_predictions, {evaluator.metricName: "r2"})

print("=" * 60)
print("GRADIENT BOOSTING MODEL EVALUATION")
print("=" * 60)
print(f"RMSE (Root Mean Squared Error): {gbt_rmse:.4f}")
print(f"MAE (Mean Absolute Error): {gbt_mae:.4f}")
print(f"R² (Coefficient of Determination): {gbt_r2:.4f}")
print("=" * 60)

GRADIENT BOOSTING MODEL EVALUATION
RMSE (Root Mean Squared Error): 19.4870
MAE (Mean Absolute Error): 10.3238
R² (Coefficient of Determination): 0.9950


In [128]:
# Evaluate Decision Tree model for comparison
dt_rmse = evaluator.evaluate(predictionsNonPipeline, {evaluator.metricName: "rmse"})
dt_mae = evaluator.evaluate(predictionsNonPipeline, {evaluator.metricName: "mae"})
dt_r2 = evaluator.evaluate(predictionsNonPipeline, {evaluator.metricName: "r2"})

print("=" * 60)
print("DECISION TREE MODEL EVALUATION")
print("=" * 60)
print(f"RMSE (Root Mean Squared Error): {dt_rmse:.4f}")
print(f"MAE (Mean Absolute Error): {dt_mae:.4f}")
print(f"R² (Coefficient of Determination): {dt_r2:.4f}")
print("=" * 60)

DECISION TREE MODEL EVALUATION
RMSE (Root Mean Squared Error): 25.5989
MAE (Mean Absolute Error): 17.9671
R² (Coefficient of Determination): 0.9914


In [129]:
# Model Comparison
print("\n" + "=" * 60)
print("MODEL COMPARISON")
print("=" * 60)
print(f"{'Metric':<30} {'Decision Tree':<20} {'Gradient Boosting':<20}")
print("-" * 60)
print(f"{'RMSE':<30} {dt_rmse:<20.4f} {gbt_rmse:<20.4f}")
print(f"{'MAE':<30} {dt_mae:<20.4f} {gbt_mae:<20.4f}")
print(f"{'R²':<30} {dt_r2:<20.4f} {gbt_r2:<20.4f}")
print("=" * 60)

# Determine which model performs better
if gbt_rmse < dt_rmse:
    print("\n✓ Gradient Boosting has lower RMSE (better)")
else:
    print("\n✓ Decision Tree has lower RMSE (better)")

if gbt_r2 > dt_r2:
    print("✓ Gradient Boosting has higher R² (better)")
else:
    print("✓ Decision Tree has higher R² (better)")


MODEL COMPARISON
Metric                         Decision Tree        Gradient Boosting   
------------------------------------------------------------
RMSE                           25.5989              19.4870             
MAE                            17.9671              10.3238             
R²                             0.9914               0.9950              

✓ Gradient Boosting has lower RMSE (better)
✓ Gradient Boosting has higher R² (better)
